In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda, Dropout, BatchNormalization, Add, GlobalAveragePooling2D, concatenate
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

In [ ]:
# Function to load images from a folder
def load_images_from_folder(folder, augmentation=False):
    images = []
    datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                                 shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest') if augmentation else None

    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = preprocess_image(img_path)
        if img is not None:
            if augmentation:
                img = np.expand_dims(img, axis=0)
                for batch in datagen.flow(img, batch_size=1):
                    images.append(batch[0])
                    break
            else:
                images.append(img)

    return np.array(images)

In [ ]:
def preprocess_image(image_path):
    # Load the image in grayscale
    img = load_img(image_path, color_mode='grayscale', target_size=(128, 128))
    img = img_to_array(img).astype('uint8')
    
    # Apply edge detection
    edges = cv2.Canny(img, threshold1=30, threshold2=100)
    
    # Dilate the edges to broaden the lines
    kernel = np.ones((3,3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)
    
    # Invert the edges: detected edges should be black, and the rest should be white
    edges = cv2.bitwise_not(edges)
    
    # Normalize the image
    edges = edges / 255.0
    edges = np.expand_dims(edges, axis=-1)
    
    return edges

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if img_path is not None:
            processed_img = preprocess_image(img_path)
            images.append(processed_img)
    return images

In [ ]:
def create_dataset(base_path):
    X = []
    y = []
    for person_id in range(1, 65):  # Assuming you have 64 persons
        if person_id in [5,7,8,10,11]:
            continue
        person_id = str(person_id).zfill(3)
        real_folder = os.path.join(base_path, person_id)
        forge_folder = os.path.join(base_path, f"{person_id}_forg")

        real_images = load_images_from_folder(real_folder)
        forge_images = load_images_from_folder(forge_folder)

        for img in real_images:
            X.append(img)
            y.append(0)  # Label for genuine signatures

        for img in forge_images:
            X.append(img)
            y.append(1)  # Label for forged signatures

    X = np.array(X)
    y = np.array(y)
    X = X / 255.0  # Normalize the images
    X = X.reshape(-1, 128, 128, 1)  # Reshape for the CNN
    return X, y

base_path = 'sign_data/train'
X, y = create_dataset(base_path)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Display a single image before and after preprocessing
def display_image_before_after(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (128, 128))
    processed_img = preprocess_image(file_path)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(img_resized, cmap='gray')
    plt.subplot(1, 2, 2)
    plt.title('Processed Image')
    plt.imshow(processed_img, cmap='gray')
    plt.show()

# Example image display
example_image_path = 'sign_data/train/001/001_01.PNG'  # Provide a valid image path here
display_image_before_after(example_image_path)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

def build_complex_model():
    model = Sequential()
    
    # First convolutional block
    model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(128, 128, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Second convolutional block
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Third convolutional block
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Fourth convolutional block
    model.add(Conv2D(1024, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Flatten and dense layers
    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_complex_model()

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    fill_mode="nearest",
    horizontal_flip=True,
    validation_split=0.2
)

# Train the model
train_generator = datagen.flow(X_train, y_train, batch_size=32, subset='training')
validation_generator = datagen.flow(X_train, y_train, batch_size=32, subset='validation')

history = model.fit(train_generator, epochs=50, validation_data=validation_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

# Save the model using Keras method
model.save('complex_signature_verification_model_improved.keras')

# Plot the model summary
plot_model(model, to_file='model_summary.png', show_shapes=True, show_layer_names=True)

# Print the model summary
model.summary()


In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# Load the saved model
model = load_model('complex_signature_verification_model123.keras')

2024-06-29 01:48:27.142860: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-29 01:48:27.163693: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
def preprocess_image1(image_path):
    processed_img = preprocess_image(image_path)
    processed_img = processed_img / 255.0
    processed_img = processed_img.reshape(1, 128, 128, 1)
    return processed_img

def predict_signature(model, real_signature_path, test_signature_path):
    real_img = preprocess_image1(real_signature_path)
    test_img = preprocess_image1(test_signature_path)
    
    real_pred = model.predict(real_img)
    test_pred = model.predict(test_img)
    
    difference = np.abs(real_pred - test_pred)
    return difference

# Example usage
real_signature_path = 'img1.jpeg'  # Provide a valid image path
test_signature_path = 'img1.jpeg'  # Provide a valid image path
difference = predict_signature(model, real_signature_path, test_signature_path)
print(f"Difference: {difference}")


In [3]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('complex_signature_verification_model123.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpg67gns4_/assets


INFO:tensorflow:Assets written to: /tmp/tmpg67gns4_/assets


: 